In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/alekseikatridi/my-mlops/2week/mlruns/1', creation_time=1685300379159, experiment_id='1', last_update_time=1685300379159, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [18]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [20]:
import xgboost as xgb

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[22:14:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.00166                                                                                                 
[1]	validation-rmse:18.89677                                                                                                 
[2]	validation-rmse:17.87316                                                                                                 
[3]	validation-rmse:16.92649                                                                                                 
[4]	validation-rmse:16.05115                                                                                                 
[5]	validation-rmse:15.24190                                                                                                 
[6]	validation-rmse:14.49534  

[63]	validation-rmse:6.59910                                                                                                 
[64]	validation-rmse:6.59491                                                                                                 
[65]	validation-rmse:6.59071                                                                                                 
[66]	validation-rmse:6.58690                                                                                                 
[67]	validation-rmse:6.58347                                                                                                 
[68]	validation-rmse:6.58044                                                                                                 
[69]	validation-rmse:6.57697                                                                                                 
[70]	validation-rmse:6.57394                                                                                          

[128]	validation-rmse:6.51112                                                                                                
[129]	validation-rmse:6.51062                                                                                                
[130]	validation-rmse:6.51029                                                                                                
[131]	validation-rmse:6.50997                                                                                                
[132]	validation-rmse:6.50974                                                                                                
[133]	validation-rmse:6.50940                                                                                                
[134]	validation-rmse:6.50916                                                                                                
[135]	validation-rmse:6.50884                                                                                         

[193]	validation-rmse:6.49787                                                                                                
[194]	validation-rmse:6.49786                                                                                                
[195]	validation-rmse:6.49777                                                                                                
[196]	validation-rmse:6.49764                                                                                                
[197]	validation-rmse:6.49748                                                                                                
[198]	validation-rmse:6.49739                                                                                                
[199]	validation-rmse:6.49716                                                                                                
[200]	validation-rmse:6.49712                                                                                         

[258]	validation-rmse:6.49117                                                                                                
[259]	validation-rmse:6.49086                                                                                                
[260]	validation-rmse:6.49081                                                                                                
[261]	validation-rmse:6.49053                                                                                                
[262]	validation-rmse:6.49047                                                                                                
[263]	validation-rmse:6.49040                                                                                                
[264]	validation-rmse:6.49024                                                                                                
[265]	validation-rmse:6.49022                                                                                         

...

[498]	validation-rmse:6.33077                                                                                                
[499]	validation-rmse:6.33069                                                                                                
[500]	validation-rmse:6.33034                                                                                                
[501]	validation-rmse:6.33006                                                                                                
[502]	validation-rmse:6.32977                                                                                                
[503]	validation-rmse:6.32954                                                                                                
[504]	validation-rmse:6.32936                                                                                                
[505]	validation-rmse:6.32941                                                                                         

[563]	validation-rmse:6.31972                                                                                                
[564]	validation-rmse:6.31945                                                                                                
[565]	validation-rmse:6.31942                                                                                                
[566]	validation-rmse:6.31937                                                                                                
[567]	validation-rmse:6.31954                                                                                                
[568]	validation-rmse:6.31930                                                                                                
[569]	validation-rmse:6.31904                                                                                                
[570]	validation-rmse:6.31865                                                                                         

[628]	validation-rmse:6.31285                                                                                                
[629]	validation-rmse:6.31274                                                                                                
[630]	validation-rmse:6.31264                                                                                                
[631]	validation-rmse:6.31274                                                                                                
[632]	validation-rmse:6.31256                                                                                                
[633]	validation-rmse:6.31263                                                                                                
[634]	validation-rmse:6.31247                                                                                                
[635]	validation-rmse:6.31225                                                                                         

[693]	validation-rmse:6.30736                                                                                                
[694]	validation-rmse:6.30735                                                                                                
[695]	validation-rmse:6.30699                                                                                                
[696]	validation-rmse:6.30694                                                                                                
[697]	validation-rmse:6.30678                                                                                                
[698]	validation-rmse:6.30664                                                                                                
[699]	validation-rmse:6.30665                                                                                                
[700]	validation-rmse:6.30659                                                                                         

[758]	validation-rmse:6.30259                                                                                                
[759]	validation-rmse:6.30260                                                                                                
[760]	validation-rmse:6.30280                                                                                                
[761]	validation-rmse:6.30260                                                                                                
[762]	validation-rmse:6.30265                                                                                                
[763]	validation-rmse:6.30254                                                                                                
[764]	validation-rmse:6.30253                                                                                                
[765]	validation-rmse:6.30252                                                                                         

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        